In [31]:
! pip install selenium
! pip install html_table_parser

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for beautifulsoup4==4.4.1 from https://files.pythonhosted.org/packages/cf/69/9abfdab06490af5e0233bcebe3f617ec128486d94ea987ad4f77b9332eef/beautifulsoup4-4.4.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/81.4 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.4 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 81.4/81.4 kB 1.5 MB/s eta 0:00:00
  Created wheel for html_table_parser: filename=html_table_parser-0.1.0-py3-none-any.whl size=4638 sha256=2f599eac9f28c620288fd47ba00512a4bafaa8e8464d32c88e500e5168094cc8
  Stored in directory: c:\users\melid\appdata\local\pip\cache\wheels\3f\61\3e\205c3503f7c244357c1ba105368ba70095a71a075624689850
Successfully built html_table_parser
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.12.2
    Uninstalling beau

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.1 which is incompatible.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

In [2]:
driver = webdriver.Chrome()

In [3]:
url = "https://statiz.sporki.com/stats/?m=main&m2=all"
driver.get(url)

In [4]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[5]/section/div[1]/div/button[2]').click()

# 팀 옵션 드롭다운 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/button').click()
# 두산 베어스 선택
driver.find_element(By.XPATH, '//*[@id="select_team"]/ul/li[5]').click()

# 규정 드롭다운 선택
driver.find_element(By.XPATH, '//*[@id="select_reg"]/button').click()
# 규정 > 전체 선택
driver.find_element(By.XPATH, '//*[@id="select_reg"]/ul/li[5]').click()

In [7]:
from urllib.request import urlopen

import collections
collections.Callable = collections.abc.Callable

from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import pandas as pd

In [8]:
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')

In [9]:
temp = soup.find_all('table')
p = parser.make2d(temp[0])    
for i in range(len(p)):
    print(i, p[i])
print(len(p))

0 ['Rank', 'Name', 'Team', 'Sort▼', 'G', 'PA', 'ePA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HP', 'IB', 'SO', 'GDP', 'SH', 'SF', '비율', '비율', '비율', '비율', '비율', '비율', 'WAR▼']
1 ['Rank', 'Name', 'Team', 'WAR', 'G', 'PA', 'ePA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HP', 'IB', 'SO', 'GDP', 'SH', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'R/ePA', 'wRC+', 'WAR▼']
2 ['1', '허경민', '24  3B', '2.46', '42', '172', '172', '147', '28', '54', '9', '0', '2', '69', '24', '1', '0', '12', '9', '0', '10', '1', '0', '4', '0.367', '0.436', '0.469', '0.905', '0.070', '150.1', '2.46']
3 ['2', '강승호', '24  2B', '2.44', '43', '185', '184', '175', '27', '59', '11', '4', '9', '105', '33', '5', '3', '8', '0', '1', '57', '2', '0', '2', '0.337', '0.362', '0.600', '0.962', '0.067', '147.5', '2.44']
4 ['3', '양의지', '24  C', '1.41', '38', '159', '158', '144', '19', '50', '6', '0', '6', '74', '37', '1', '0', '9', '2', '1', '18', '3', '0', '4', '0.347', '0.384', '0.514', '0.89

In [10]:
num = (len(p) // 10)-1
start_num = 2
for i in range(num*2+1, 1, -1):
    n = start_num * 10 + (i)
    p.pop(n)
    if i % 2 == 0:
        start_num -= 1

In [15]:
data=pd.DataFrame(p)
data = data[2:]
data.columns=["Rank","Name","Position","WAR","G","PA","ePA","AB","R","H","2B","3B","HR","TB","RBI","SB","CS","BB","HP","IB","SO","GDP","SH","SF","AVG","OBP","SLG","OPS","R/ePA","wRC+","WAR2"]
data['Position'].replace({"24  " : ""}, regex=True,inplace=True)
data['Team']="Doosan"
data.drop(columns=["WAR2"], inplace=True)
print(data)

   Rank Name Position    WAR   G   PA  ePA   AB   R   H  ... GDP SH SF    AVG  \
2     1  허경민       3B   2.46  42  172  172  147  28  54  ...   1  0  4  0.367   
3     2  강승호       2B   2.44  43  185  184  175  27  59  ...   2  0  2  0.337   
4     3  양의지        C   1.41  38  159  158  144  19  50  ...   3  0  4  0.347   
5     4  조수행       CF   0.99  36   92   87   82  19  26  ...   1  5  1  0.317   
6     5  라모스       RF   0.97  30  127  127  113  17  34  ...   1  0  5  0.301   
7     6  박준영       SS   0.95  34  118  117   99  16  21  ...   0  1  1  0.212   
8     7  정수빈       CF   0.73  42  190  186  159  31  44  ...   4  4  3  0.277   
9     8  이유찬       2B   0.34  17    8    8    8   5   4  ...   0  0  0  0.500   
10    9  김재환       LF   0.32  41  168  166  139  24  33  ...   5  0  3  0.237   
11   10  김기연        C   0.31  18   52   52   49   6  18  ...   3  0  0  0.367   
12   11  양석환       1B   0.27  43  174  174  153  22  37  ...  10  0  1  0.242   
13   12  김민혁       1B   0.25

In [16]:
data.to_csv("Doosan_hit.csv", index=False)